# Local reliability, RMS violated constraints: evaluate

Evaluate local reliability for 1, 2 and 3 params, using the RMS violated inequality constraints as the requirement violation severity metric. This notebook has been run multiple times, starting with just `fimp_14`, then adding `psepbqarmax` and finally `triang`, each time producing a different campaign name to distinguish the campaign databases.

In [1]:
import easyvvuq as uq
import chaospy as cp
from pathlib import Path
from infeas.decoder import MfileDecoder

In [2]:
# Define campaign
WORK_DIR = "campaigns"
Path("campaigns").mkdir(exist_ok=True)
campaign = uq.Campaign(name="3p_only", work_dir=WORK_DIR)

# Define parameter space
# Uncertainties from Alex's SA paper

params = {
    # Default values are at solution point
    "fimp_14": {
        "type": "float",
        "min": 5.0e-6,
        "max": 1.0e-4,
        "default": 5.0e-6,
    },  # ok
    "psepbqarmax": {"type": "float", "min": 8.7, "max": 10.0, "default": 10.0},  # ok
    "triang": {"type": "float", "min": 0.4, "max": 0.6, "default": 0.5},  # ok
}

# QoIs
# Violated constraint residuals
qois = [
    "rms_vio_constr_res",
]

Next, define a custom decoder to handle Process's mfile output.

In [3]:
# Create encoder and decoder
encoder = uq.encoders.GenericEncoder(
    template_fname="large_tokamak_sol_IN_DAT.template", target_filename="IN.DAT"
)

decoder = MfileDecoder(target_filename="MFILE.DAT", output_columns=qois)

cmd = "process -i IN.DAT"
actions = uq.actions.local_execute(encoder, cmd, decoder)

# Add the app
campaign.add_app(name="feasibility", params=params, actions=actions)

In [4]:
# Create PCE sampler
# Vary 3 uncertain inputs
vary = {
    "fimp_14": cp.Uniform(
        1.0e-5,
        1.0e-4,
        # Soln value: 5.0e-6: outside bounds
    ),
    "psepbqarmax": cp.Uniform(
        8.7,
        10.0,
    ),
    "triang": cp.Uniform(
        0.4,
        0.6,
    ),
}
sampler = uq.sampling.PCESampler(vary=vary, polynomial_order=4)
# Optionally, try MC
# sampler = uq.sampling.MCSampler(vary=vary, n_mc_samples=125)
# Actually 625 samples in practice: Saltelli method

# Add sampler to campaign
campaign.set_sampler(sampler)

# Draw samples, execute and collate
campaign.execute().collate(progress_bar=True)
samples = campaign.get_collation_result()

  0%|          | 0/625 [00:00<?, ?it/s]

The IN.DAT file does not contain any obsolete variables.
 tmargmin_cs and tmargmin should not both be specified in IN.DAT.
 tmargmin_cs has been ignored.
 
 **************************************************************************************************************
 ************************************************** PROCESS ***************************************************
 ************************************** Power Reactor Optimisation Code ***************************************
 **************************************************************************************************************
 
   Program :
   Version : 3.0.2   Release Date :: 2024-01-25
   Tag No. : v3.0.2-24-g00e1700
    Branch : main
   Git log : Added check to see if fimp(2) deviates from default value, if it does… ([hash]3053)
 Date/time : 27 Feb 2024 11:53:37 +00:00(hh:mm) UTC
      User : jon
  Computer : jon-Precision-3560
 Directory :
 Warning in routine OCMMNT :
    Input : /home/jon/code/easyVVUQ-process

Traceback (most recent call last):
  File "/home/jon/anaconda3/envs/easyVVUQ-process/bin/process", line 33, in <module>
    sys.exit(load_entry_point('process', 'console_scripts', 'process')())
  File "/home/jon/code/process/process/main.py", line 650, in main
    Process(args)
  File "/home/jon/code/process/process/main.py", line 127, in __init__
    self.run_mode()
  File "/home/jon/code/process/process/main.py", line 204, in run_mode
    self.run.run()
  File "/home/jon/code/process/process/main.py", line 363, in run
    self.run_scan(self.solver)
  File "/home/jon/code/process/process/main.py", line 475, in run_scan
    final.finalise(self.models, self.ifail)
  File "/home/jon/code/process/process/final.py", line 23, in finalise
    op.write(models, ft.constants.nout)
  File "/home/jon/code/process/process/output.py", line 44, in write
    models.physics.outplas()
  File "/home/jon/code/process/process/physics.py", line 4239, in outplas
    f"Confinement scaling law: { physics_vari

The IN.DAT file does not contain any obsolete variables.
 tmargmin_cs and tmargmin should not both be specified in IN.DAT.
 tmargmin_cs has been ignored.
 
 **************************************************************************************************************
 ************************************************** PROCESS ***************************************************
 ************************************** Power Reactor Optimisation Code ***************************************
 **************************************************************************************************************
 
   Program :
   Version : 3.0.2   Release Date :: 2024-01-25
   Tag No. : v3.0.2-24-g00e1700
    Branch : main
   Git log : Added check to see if fimp(2) deviates from default value, if it does… ([hash]3053)
 Date/time : 27 Feb 2024 11:53:38 +00:00(hh:mm) UTC
      User : jon
  Computer : jon-Precision-3560
 Directory :
 Warning in routine OCMMNT :
    Input : /home/jon/code/easyVVUQ-process

Traceback (most recent call last):
  File "/home/jon/anaconda3/envs/easyVVUQ-process/bin/process", line 33, in <module>
    sys.exit(load_entry_point('process', 'console_scripts', 'process')())
  File "/home/jon/code/process/process/main.py", line 650, in main
    Process(args)
  File "/home/jon/code/process/process/main.py", line 127, in __init__
    self.run_mode()
  File "/home/jon/code/process/process/main.py", line 204, in run_mode
    self.run.run()
  File "/home/jon/code/process/process/main.py", line 363, in run
    self.run_scan(self.solver)
  File "/home/jon/code/process/process/main.py", line 475, in run_scan
    final.finalise(self.models, self.ifail)
  File "/home/jon/code/process/process/final.py", line 23, in finalise
    op.write(models, ft.constants.nout)
  File "/home/jon/code/process/process/output.py", line 44, in write
    models.physics.outplas()
  File "/home/jon/code/process/process/physics.py", line 4239, in outplas
    f"Confinement scaling law: { physics_vari

...

100%|█████████▉| 622/625 [05:10<00:00,  3.02it/s]

The IN.DAT file does not contain any obsolete variables.
 tmargmin_cs and tmargmin should not both be specified in IN.DAT.
 tmargmin_cs has been ignored.
 
 **************************************************************************************************************
 ************************************************** PROCESS ***************************************************
 ************************************** Power Reactor Optimisation Code ***************************************
 **************************************************************************************************************
 
   Program :
   Version : 3.0.2   Release Date :: 2024-01-25
   Tag No. : v3.0.2-24-g00e1700
    Branch : main
   Git log : Added check to see if fimp(2) deviates from default value, if it does… ([hash]3053)
 Date/time : 27 Feb 2024 11:58:46 +00:00(hh:mm) UTC
      User : jon
  Computer : jon-Precision-3560
 Directory :
 Warning in routine OCMMNT :
    Input : /home/jon/code/easyVVUQ-process

Traceback (most recent call last):
  File "/home/jon/anaconda3/envs/easyVVUQ-process/bin/process", line 33, in <module>
    sys.exit(load_entry_point('process', 'console_scripts', 'process')())
  File "/home/jon/code/process/process/main.py", line 650, in main
    Process(args)
  File "/home/jon/code/process/process/main.py", line 127, in __init__
    self.run_mode()
  File "/home/jon/code/process/process/main.py", line 204, in run_mode
    self.run.run()
  File "/home/jon/code/process/process/main.py", line 363, in run
    self.run_scan(self.solver)
  File "/home/jon/code/process/process/main.py", line 475, in run_scan
    final.finalise(self.models, self.ifail)
  File "/home/jon/code/process/process/final.py", line 23, in finalise
    op.write(models, ft.constants.nout)
  File "/home/jon/code/process/process/output.py", line 44, in write
    models.physics.outplas()
  File "/home/jon/code/process/process/physics.py", line 4239, in outplas
    f"Confinement scaling law: { physics_vari

The IN.DAT file does not contain any obsolete variables.
 tmargmin_cs and tmargmin should not both be specified in IN.DAT.
 tmargmin_cs has been ignored.
 
 **************************************************************************************************************
 ************************************************** PROCESS ***************************************************
 ************************************** Power Reactor Optimisation Code ***************************************
 **************************************************************************************************************
 
   Program :
   Version : 3.0.2   Release Date :: 2024-01-25
   Tag No. : v3.0.2-24-g00e1700
    Branch : main
   Git log : Added check to see if fimp(2) deviates from default value, if it does… ([hash]3053)
 Date/time : 27 Feb 2024 11:58:47 +00:00(hh:mm) UTC
      User : jon
  Computer : jon-Precision-3560
 Directory :
 Warning in routine OCMMNT :
    Input : /home/jon/code/easyVVUQ-process

Traceback (most recent call last):
  File "/home/jon/anaconda3/envs/easyVVUQ-process/bin/process", line 33, in <module>
    sys.exit(load_entry_point('process', 'console_scripts', 'process')())
  File "/home/jon/code/process/process/main.py", line 650, in main
    Process(args)
  File "/home/jon/code/process/process/main.py", line 127, in __init__
    self.run_mode()
  File "/home/jon/code/process/process/main.py", line 204, in run_mode
    self.run.run()
  File "/home/jon/code/process/process/main.py", line 363, in run
    self.run_scan(self.solver)
  File "/home/jon/code/process/process/main.py", line 475, in run_scan
    final.finalise(self.models, self.ifail)
  File "/home/jon/code/process/process/final.py", line 23, in finalise
    op.write(models, ft.constants.nout)
  File "/home/jon/code/process/process/output.py", line 44, in write
    models.physics.outplas()
  File "/home/jon/code/process/process/physics.py", line 4239, in outplas
    f"Confinement scaling law: { physics_vari